In [ ]:
%cd ..

In [ ]:
# Import necessary modules
import numpy as np
import gzip
import matplotlib.pyplot as plt

from pbh import PrimordialBlackHole
from distribution import P_c

In [ ]:
# Define parameters
spacetime = "Kerr"
initial_mass = 5.
final_mass = 1.
initial_momentum = 0.
save_path = False

In [ ]:
# Create PrimordialBlackHole object
pbh = PrimordialBlackHole(
    spacetime,
    initial_mass, final_mass,
    initial_momentum,
    save_path=save_path,
)

In [ ]:
# Parameters of the study
N_extremal = 2000000000 # Number of total extremal black holes
N_per_batch = 100000000 # Number of extremal black holes per batch

# Calculate the number of batches needed
N_batch = int(np.ceil(N_extremal / N_per_batch))
zfl = len(str(N_batch - 1))

In [ ]:
# Initialize variables
n_total = 0
i_batch = 0

# Loop through batches
for i in range(N_batch):
    n = 0
    batch_results = np.empty((N_per_batch, 8))

    # Evolve black holes until desired number of extremal black holes reached
    while n < N_per_batch:
        pbh.evolve()
        # Check if black hole becomes extremal
        if pbh.extremal == 1:
            batch_results[n, :] = np.array([
                pbh.M_init,
                pbh.J_init,
                pbh.M_end,
                pbh.J_end, 
                pbh.a_star_end,
                pbh.n_steps,
                pbh.extremal,
                pbh.computation_time,
            ])
            n += 1
            # Check if it's the last batch
            if i == N_batch - 1:
                if n_total + n == N_extremal:
                    # Adjust batch size
                    batch_results = batch_results[:n]
                    break
    n_total += n

    # Save batch results
    f = gzip.GzipFile(f"data/extremal_batch_{i:0{zfl}}.npy.gz", 'w')
    np.save(file=f, arr=batch_results)

In [ ]:
# Load extremal masses
extremal_masses = np.empty(N_extremal)
for i in range(N_batch):
    f = gzip.GzipFile(f"data/extremal_batch_{i:0{zfl}}.npy.gz", 'r')
    data = np.load(f)
    extremal_masses[
        i * N_per_batch : i * N_per_batch + data.shape[0]
    ] = data[:, 2]

In [ ]:
# Calculate histogram bins and data
int_vals = np.sqrt(np.arange(1, 9 + 1e-10, 1))
bins = np.array([])
nums = (np.arange(2, 10, 1)**2)[::-1]

for i in range(int_vals.size-1):
    int_bins = np.linspace(
        int_vals[i], int_vals[i+1], num=nums[i], endpoint=False,
    )
    bins = np.append(bins, int_bins)

bins = np.append(bins, 3)

hist = np.histogram(extremal_masses, bins=bins, density=True)

In [ ]:
# Calculate probability distribution
xs = []
results = []

for i in range(int_vals.size-1):
    int_bins = np.linspace(
        int_vals[i], int_vals[i+1], num=100, endpoint=True,
    )
    int_bins[0] += 1e-8
    int_bins[-1] -= 1e-8
    xs.append(int_bins)
    results.append(P_c(int_bins))

In [ ]:
# Plot histogram and probability distribution
plt.figure()

plt.bar(bins[:-1], hist[0], width=np.diff(bins), align="edge")

for x, y in zip(xs, results):
    plt.plot(x, y, c='r')

plt.yscale('log')

plt.xlabel(r"extremal mass $M_{\mathrm{ext}}$")
plt.ylabel(r"probability distribution $P_{\mathrm{c}}(M_{\mathrm{ext}})$")

plt.show()